# Lecture 3: Attack on AES with Differential-Power-Analysis (Kocher et al. 1999)


#### Learning goals
- Recap how AES works
- Learn how to perform a _Differential-Power-Analysis_ by Kocher et al. 1999
- Learn how to partition traces

#### References
- https://paulkocher.com/doc/DifferentialPowerAnalysis.pdf
- https://link.springer.com/article/10.1007/s13389-011-0006-y

In this example we want to attack an AES encryption using the _Differential-Power-Analysis_ introduces by Kocher et al. 1999.

The Differential-Power-Analysis has few prerequisites or in other words: It can be applied if the following conditions hold true:
- The device holds a **secret** (e.g. a key)
- The **secret is constant** during all measurements
- External **varying input** can be fed into the device
- The secret "**collides**" with external input

Notes:
- "Collide" means: The input and the secret are processed together in an algorithm
- Varying input: It is not required that the input can be determined by the attacker. It is sufficient that the attacker knows the input and the input is sufficiently random.
- If the input is unknown but varying the attack also works if the output is known by the attacker.

## 1. Understand AES and capture traces from a Sbox Lookup

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

- Recap the steps of an AES encryption (e.g. https://de.wikipedia.org/wiki/Advanced_Encryption_Standard)
- Why is _SubBytes_ a "good" place to mount an attack?

</div>

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

The file [./sbox_lookup.c](./sbox_lookup.c) contains the essentials of an AES SBox lookup, i.e. the step SubBytes in an AES encryption.

1. Record one trace using `input=lambda _: 16 * [0]`.
2. Plot the trace. What can you see? Any repeating patterns?
3. Open the corresponding assembly listing file [generic_simpleserial-CWLITEXMEGA.lss](./build/generic_simpleserial-CWLITEXMEGA.lss) and understand the instructions in the function `run`.

</div>

## 2. Develop an attack

The term _Differential_ in Differential-Power-Analysis is not chosen by accident.
We will use differences to develop an attack that reveals the secret key.

#### Basic principle of a Differential-Power-Analysis

1. Divide the set of traces (with random input) into **two partitions** using a **guess**.
2. Calculate the average of each partition.
3. Calculate the **absolute difference** of the two averages.
4. The maximal difference corresponds to the correct guess.

But what does "partition using a guess" mean?

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

In this exercise we don't want to reveal the key but understand why differences are good to reveal data.

1. Record 1000 traces using random input for the first input byte, i.e. use `input=lambda _: [random.randint(0, 255)] + 15 * [0]`.
2. Divide the data into two sets `data_msb_0` and `data_msb_1` depending on the msb of the random byte. </br>
   Hints:
   - The first byte of `input` of trace `i` can be accessed using `data[i]["input"][0]`.
   - The MSB of a byte `x` can be calculated using `x & 0x80`
   - A set can be created using Python's list comprehension: `[d for d in data if msb_first_byte_of_input(d) == 0]`

3. Create an numpy array of the result: `data_msb_0 = np.array(data_msb_0)`, `data_msb_1 = np.array(data_msb_1)`
4. Calculate the "mean-trace" of each set: `mean_trace_msb_0 = np.mean(data_msb_0["trace"], axis=0)`, `mean_trace_msb_1 = np.mean(data_msb_1["trace"], axis=0)`.
   Lean back and think about the result. </br>
   Hints:
   - The dimensions of a numpy array can be shown using `data.shape`

5. Plot the absolute difference between the two mean-traces.
   What do you see and why?

   If you can explain, you understood one main principle of DPA! 💪

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

In Exercise 3 the input was revealed using differences.
But the input is usually not of interest as we already know it.
In this exercise you will learn the second main principle of DPA: Guessing!

1. Use the traces from Exercise 3 and divide them into two partitions using the following selection function:

   "Guessing the first key byte to `0x00`, select all traces where the MSB of the output of the first SBox lookup is `0`."

   The other partition is defined by the rest.
2. Repeat steps 3-5 of Exercise 3 with the new partition.
   You won't see anything interesting 😉. 
   Continue with the next Exercise!

Hints:
- The AES SBox can be imported with: `from lascar.tools.aes import sbox`

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 5</div>

Repeat Exercise 4 with key guess `0x01`. Can you see any difference? Why?

How can this be evolved to an attack?

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 6</div>

Implement an automated attack using Differential-Power-Analysis, i.e. implement a function with the following signature:

```python
def aes_sbox_dpa(
    traces,
    key_byte_index=0,
):
```

Hints:
- Use the MSB of the SBox output

Optional task:
- Introduce an additional input parameter, `selection_bit_index`, where the bit, which is selected for partitioning, can be varied.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 7</div>

Change the key in [./sbox_lookup.c](./sbox_lookup.c), compile, and flash. Exchange with another student and challenge your attack function.

</div>